In [ ]:
dk_datasets =[
    {
        "dataset_name": "hestenet",
        "dataset_hf_path": "kardosdrur/hestenet-qa",
        "query": "question",
        "pos": "answer",
        "loss": "multiple_negatives_ranking",
        "label": ""
    },
    {
        "dataset_name": "folketinget",
        "dataset_hf_path": "kardosdrur/folketinget-discussions",
        "query": "comment",
        "pos": "response",
        "loss": "multiple_negatives_ranking",
        "label": ""
    },
    {
        "dataset_name": "wiki_qa",
        "dataset_hf_path": "kardosdrur/dawiki_qa_zephyr",
        "query": "question",
        "pos": "answer",
        "loss": "multiple_negatives_ranking",
        "label": ""
    },
    {
        "dataset_name": "news",
        "dataset_hf_path": "kardosdrur/danews_title_content",
        "query": "title",
        "pos": "content",
        "loss": "multiple_negatives_ranking",
        "label": ""
    },
    {
        "dataset_name": "opensubtitles_da_no",
        "dataset_hf_path": "kardosdrur/opensubtitles-no-da",
        "query": "no",
        "pos": "da",
        "loss": "multiple_negatives_ranking",
        "label": "overlap"
    },
    {
        "dataset_name": "europarl",
        "dataset_hf_path": "kardosdrur/europarl-scandinavian",
        "query": "sv",
        "pos": "da",
        "loss": "multiple_negatives_ranking",
        "label": ""
    },
        "dataset_name": "wiki_queries_gemma",
        "dataset_hf_path": "DDSC/da-wikipedia-queries-gemma-processed",
        "query": "anchor",
        "pos": "positive",
        "loss": "multiple_negatives_ranking",
        "label": ""
]

In [1]:
dk_datasets =[
    {    "dataset_name": "wiki_queries_gemma",
        "dataset_hf_path": "DDSC/da-wikipedia-queries-gemma-processed",
        "query": "anchor",
        "pos": "positive",
        "loss": "multiple_negatives_ranking",
        "label": ""
    }
]

In [2]:
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict

def create_combined_dataset(json_list):
    combined_data = []

    for entry in json_list:
        # Attempt to load only the 'train' split; handle cases where the dataset split might not be directly specified
        try:
            dataset = load_dataset(entry['dataset_hf_path'], split='train')
        except ValueError:
            # Default to the first available split if 'train' is not explicitly available; you can handle as needed
            dataset_dict = load_dataset(entry['dataset_hf_path'])
            if 'train' in dataset_dict:
                dataset = dataset_dict['train']
            else:
                # If 'train' is not available specifically, choose the first available
                first_split = next(iter(dataset_dict.keys()))
                dataset = dataset_dict[first_split]

        # Select only the relevant columns and limit to 100,000 samples
        truncated_dataset = dataset.select(range(min(100000, len(dataset)))).map(
            lambda example: {
                'query': example[entry['query']],
                'pos': example[entry['pos']],
                'dataset_name': entry['dataset_name']
            },
            remove_columns=dataset.column_names
        )

        # Append each processed dataset to the list
        combined_data.append(truncated_dataset)

    # Concatenate all datasets into one
    combined_dataset = concatenate_datasets(combined_data)

    return combined_dataset

In [3]:
combined_dataset = create_combined_dataset(dk_datasets)

README.md:   0%|          | 0.00/653 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30280 [00:00<?, ? examples/s]

Map:   0%|          | 0/30280 [00:00<?, ? examples/s]

In [4]:
combined_dataset

Dataset({
    features: ['query', 'pos', 'dataset_name'],
    num_rows: 30280
})

In [7]:
# Push the directory with the multiple configurations to the Hub
combined_dataset.push_to_hub(repo_id="jealk/supervised-da", config_name="default", set_default=True, commit_message="1M samples from 6 Danish datasets for supervised embedding finetuning")

NameError: name 'combined_dataset' is not defined

In [5]:
from datasets import Dataset
import os
import json

def save_combined_dataset_as_jsonl(combined_dataset, save_directory):
    os.makedirs(save_directory, exist_ok=True)
    grouped_samples = {}

    for sample in combined_dataset:
        dataset_name = sample['dataset_name']
        
        if dataset_name not in grouped_samples:
            grouped_samples[dataset_name] = []
        
        # Prepare the sample
        json_sample = {
            "query": sample['query'],
            "positive": sample['pos'],
            "negative": ""  # Placeholder: you might have logic to set this
                            # if there are negative examples as well
        }
        
        grouped_samples[dataset_name].append(json_sample)
    
    for dataset_name, samples in grouped_samples.items():
        jsonl_filepath = os.path.join(save_directory, f"{dataset_name}.jsonl")
        
        with open(jsonl_filepath, 'w', encoding='utf-8') as f:
            for sample in samples:
                f.write(json.dumps(sample) + "\n")
                
    print(f"Datasets saved in {save_directory} as JSONL files.")

# Example usage
# Assuming `combined_dataset` is your Hugging Face dataset object after processing
save_directory = "cache/echo-data"
save_combined_dataset_as_jsonl(combined_dataset, save_directory)

Datasets saved in cache/echo-data as JSONL files.
